In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from arguments import Arguments
from cnn import CNN
from collections import defaultdict
from distributor import get_cluster_sizes, get_distributed_data, \
    get_distributed_data_using_loader, get_fog_graph
from train import fog_train as train
from numpy import array
from numpy.random import permutation, randint
import os
import pickle as pkl
from sklearn.model_selection import train_test_split
import syft as sy
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from train import fog_train as train, test


In [ ]:
# Setups
args = Arguments()
hook = sy.TorchHook(torch)
USE_CUDA = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
device = torch.device("cuda" if USE_CUDA else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}
kwargs = {}

In [ ]:
ckpt_path = '../ckpts'
dataset = 'mnist'
clf_type = 'cnn'
paradigm = 'fog_uniform_non_iid_2_num_workers_{}_lr_{}_batch_{}'.format(
    args.num_workers,
    args.lr,
    args.batch_size
)
model_name = '{}_{}_{}'.format(dataset, clf_type, paradigm)
print(model_name)
init_path = os.path.join(ckpt_path, 'mnist_cnn_fl.init')
best_path = os.path.join(ckpt_path, model_name + '.best')
stop_path = os.path.join(ckpt_path, model_name + '.stop')

In [ ]:
# prepare graph and data
fog_graph, workers = get_fog_graph(hook, args.num_workers, args.num_clusters,
                                 args.shuffle_workers, args.uniform_clusters)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

X_trains, y_trains = pkl.load(open('../ckpts/data_non_iid_2_stratify_True_uniform_True.pkl', 'rb'))

fog_graph

In [ ]:
best = 0
# Fire the engines
model = CNN().to(device)
if args.load_init:
    model.load_state_dict(torch.load(init_path))
    print('Load init: {}'.format(init_path))
elif args.save_init:
    torch.save(model.state_dict(), init_path)
    print('Save init: {}'.format(init_path))


In [ ]:
best = 0

x_ax = []
y_ax = []
for epoch in range(1, args.epochs + 1):
    train(args, model, fog_graph, workers, X_trains, y_trains,
          device, epoch)
    acc = test(args, model, device, test_loader, best, epoch)
    y_ax.append(acc)
    x_ax.append(epoch)

    if args.save_model and acc > best:
        best = acc
        torch.save(model.state_dict(), best_path)
        print('Model best  @ {}, acc {}: {}\n'.format(epoch, acc, best_path))
    
if (args.save_model):
    torch.save(model.state_dict(), stop_path)
    print('Model stop: {}'.format(stop_path))


In [ ]:
pkl.dump((x_ax, y_ax), open('../ckpts/history_{}.pkl'.format(model_name), 'wb')) 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x_ax, y_ax)
plt.savefig('../plots/{}.png'.format(model_name))

In [ ]:
import numpy as np
[np.bincount(_) for _ in y_trains]
